# 02 Matmul**FreeCodeCamp CUDA Course - Module 5**Original Course: [https://www.youtube.com/watch?v=86FAWCzIe_4](https://www.youtube.com/watch?v=86FAWCzIe_4)Source File: `02 matmul.cu`---## OverviewImplement matrix multiplication using CUDA.---

## Learning ObjectivesBy the end of this notebook, you will:1. Understand CUDA kernel syntax and execution2. Learn GPU memory allocation and data transfer---

## SetupMake sure you've completed the setup from the first notebook (GPU enabled, nvcc4jupyter installed).---

## Key Concepts- **Kernel Function**: Uses `__global__` qualifier for GPU execution- **Device Memory**: Allocated using `cudaMalloc`- **Data Transfer**: Uses `cudaMemcpy` between host and device- **Kernel Launch**: Syntax `kernel<<<blocks, threads>>>(...)`- **Synchronization**: `cudaDeviceSynchronize()` waits for GPU completion---## CUDA Implementation

In [ ]:
%%cu#include <stdio.h>#include <stdlib.h>#include <time.h>#include <cuda_runtime.h>#define M 256  // Number of rows in A and C#define K 512   // Number of columns in A and rows in B#define N 256  // Number of columns in B and C#define BLOCK_SIZE 32// Example 3x2 @ 2x4 = 3x4 -> (M x K) @ (K x N) = (M x N)// A = [[1, 2], //      [3, 4], //      [5, 6]]// B = [[7, 8, 9, 10],//      [11, 12, 13, 14]]// C = A * B = [[1*7 + 2*11, 1*8 + 2*12, 1*9 + 2*13, 1*10 + 2*14],//              [3*7 + 4*11, 3*8 + 4*12, 3*9 + 4*13, 3*10 + 4*14],//              [5*7 + 6*11, 5*8 + 6*12, 5*9 + 6*13, 5*10 + 6*14]]// C = [[29, 32, 35, 38],//      [65, 72, 79, 86],//      [101, 112, 123, 134]]// CPU matrix multiplicationvoid matmul_cpu(float *A, float *B, float *C, int m, int k, int n) {    for (int i = 0; i < m; i++) {        for (int j = 0; j < n; j++) {            float sum = 0.0f;            for (int l = 0; l < k; l++) {                sum += A[i * k + l] * B[l * n + j];            }            C[i * n + j] = sum;        }    }}// CUDA kernel for matrix multiplication__global__ void matmul_gpu(float *A, float *B, float *C, int m, int k, int n) {    int row = blockIdx.y * blockDim.y + threadIdx.y;    int col = blockIdx.x * blockDim.x + threadIdx.x;    if (row < m && col < n) {        float sum = 0.0f;        for (int l = 0; l < k; l++) {            sum += A[row * k + l] * B[l * n + col];        }        C[row * n + col] = sum;    }}// Initialize matrix with random valuesvoid init_matrix(float *mat, int rows, int cols) {    for (int i = 0; i < rows * cols; i++) {        mat[i] = (float)rand() / RAND_MAX;    }}// Function to measure execution timedouble get_time() {    struct timespec ts;    clock_gettime(CLOCK_MONOTONIC, &ts);    return ts.tv_sec + ts.tv_nsec * 1e-9;}int main() {    float *h_A, *h_B, *h_C_cpu, *h_C_gpu;    float *d_A, *d_B, *d_C;    int size_A = M * K * sizeof(float);    int size_B = K * N * sizeof(float);    int size_C = M * N * sizeof(float);    // Allocate host memory    h_A = (float*)malloc(size_A);    h_B = (float*)malloc(size_B);    h_C_cpu = (float*)malloc(size_C);    h_C_gpu = (float*)malloc(size_C);    // Initialize matrices    srand(time(NULL));    init_matrix(h_A, M, K);    init_matrix(h_B, K, N);    // Allocate device memory    cudaMalloc(&d_A, size_A);    cudaMalloc(&d_B, size_B);    cudaMalloc(&d_C, size_C);    // Copy data to device    cudaMemcpy(d_A, h_A, size_A, cudaMemcpyHostToDevice);    cudaMemcpy(d_B, h_B, size_B, cudaMemcpyHostToDevice);    // Define grid and block dimensions    dim3 blockDim(BLOCK_SIZE, BLOCK_SIZE);    dim3 gridDim((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (M + BLOCK_SIZE - 1) / BLOCK_SIZE);    // Warm-up runs    printf("Performing warm-up runs...\n");    for (int i = 0; i < 3; i++) {        matmul_cpu(h_A, h_B, h_C_cpu, M, K, N);        matmul_gpu<<<gridDim, blockDim>>>(d_A, d_B, d_C, M, K, N);        cudaDeviceSynchronize();    }    // Benchmark CPU implementation    printf("Benchmarking CPU implementation...\n");    double cpu_total_time = 0.0;    for (int i = 0; i < 20; i++) {        double start_time = get_time();        matmul_cpu(h_A, h_B, h_C_cpu, M, K, N);        double end_time = get_time();        cpu_total_time += end_time - start_time;    }    double cpu_avg_time = cpu_total_time / 20.0;    // Benchmark GPU implementation    printf("Benchmarking GPU implementation...\n");    double gpu_total_time = 0.0;    for (int i = 0; i < 20; i++) {        double start_time = get_time();        matmul_gpu<<<gridDim, blockDim>>>(d_A, d_B, d_C, M, K, N);        cudaDeviceSynchronize();        double end_time = get_time();        gpu_total_time += end_time - start_time;    }    double gpu_avg_time = gpu_total_time / 20.0;    // Print results    printf("CPU average time: %f microseconds\n", (cpu_avg_time * 1e6f));    printf("GPU average time: %f microseconds\n", (gpu_avg_time * 1e6f));    printf("Speedup: %fx\n", cpu_avg_time / gpu_avg_time);    // Free memory    free(h_A);    free(h_B);    free(h_C_cpu);    free(h_C_gpu);    cudaFree(d_A);    cudaFree(d_B);    cudaFree(d_C);    return 0;}

## ExercisesExperiment with these modifications:1. **Matrix Sizes**: Try different M, N, K dimensions   - What happens with non-square matrices?   - Test with very large matrices2. **Verify Correctness**: Add code to verify the result against CPU computation3. **Block Size Impact**: Experiment with different BLOCK_SIZE values   - Measure performance for each4. **Measure Throughput**: Calculate FLOPS (floating-point operations per second)

---## Key Takeaways- CUDA enables massive parallelism for compute-intensive tasks- Proper memory management is crucial for performance- Understanding the thread hierarchy helps write efficient kernels- Always synchronize when needed to ensure correctness---## Next StepsContinue to the next notebook in Module 5 to learn more CUDA concepts!---## Notes*Use this space for your learning notes:*